# Inversores

In [32]:
from z3 import *
import numpy as np

In [43]:
class Inverter:
    def __init__(self, sys_id, inv_id):
        self.input = Int(f"in_s{sys_id}_i{inv_id}")
        self.out = Int(f"out_s{sys_id}_i{inv_id}")
        
    def initstate(self):
        # Condição inicialização do estado
        r = self.out == np.random.randint(2)
        
        return r

class System:
    def __init__(self, sys_id, num_inv=4):
        self.invlist = [Inverter(sys_id, i) for i in range(num_inv)]
        
    def initsystem(self):
        # Adicionar condições das variáveis "state" e "out" dos inversores
        c1 = And([inv.initstate() for inv in self.invlist])
        
        # Adicionar condições das variáveis "in" dos inversores
        c2 = And([self.invlist[i].input == self.invlist[i-1].out for i in range(len(self.invlist))])
        
        # Juntar as duas condições anteriores
        r = And(c1, c2)
        
        return r
    

def invert(inv_atual, inv_ant):
    c1 = inv_atual.out == inv_ant.out
    c2 = inv_atual.out == 1 - inv_ant.input
    r = Or(c1, c2)
    
    return r
    
    
def trans(sys_atual, sys_ant):
    # Restrições da determinação do output de cada inversor
    conds = [sys_atual.invlist[i].input == sys_atual.invlist[i-1].out for i in range(len(sys_atual.invlist))]
    for inv_atual, inv_ant in zip(sys_atual.invlist, sys_ant.invlist):
        conds.append(invert(inv_atual, inv_ant))
    
    r = And(conds)
    
    return r


def has_ended(sys):
    # Verificar se todos os inversores estão a zero (sistema termina)
    r = And([inv.out == 0 for inv in sys.invlist])
    return r

In [44]:
def eventually_ends(K):
    r = False

    for k in range(1, K+1):
        # Criar o solver e o traço de estados temporais
        solver = Solver()
        states = {i: System(i) for i in range(k)}

        # Inicializar o primeiro estado temporal
        solver.add(states[0].initsystem())

        # Fazer a transição de estados
        for i in range(1, k):
            solver.add(trans(states[i], states[i-1]))

        for s in states:
            solver.add(Not(has_ended(states[s])))

        #if solver.check() == sat:
        #    print("The system does not end")
        #    m = solver.model()
        #    print(m)
        #else:
        if solver.check() == unsat:
            r = True
            break
            
    return r

In [49]:
eventually_ends(10)

False